# Predicting peptide antibiofilm potential

For now, we are just reproducing the work of Bose et al. 2022.
Code and data available at https://github.com/davidanastasiu/antibiofilm

In [2]:
# Packages for analysis
import pandas as pd
import numpy as np
import subprocess
import sys
import os
from pathlib import Path

# Pickle package
import pickle

# Packages for visuals
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(font_scale=1.2)
# Allows charts to appear in the notebook
%matplotlib inline

from sklearn import svm, datasets
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neural_network import MLPClassifier

from sklearn.ensemble import RandomForestClassifier


from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

path = Path.cwd() #go search for a path

# Dataset

In [3]:

peptides = pd.read_csv('peptides-all.csv', encoding='cp1252') #latin encoding
peptides
# a total of 2662 peptides, i.e. 242 positive examples and 2420 negative examples
#more negative samples taht positive
#capacity antibiofilm type=1

,Name,Type,Seq,SeqL,MwWt,Aromaticity,isoelectric point,A,R,N,...,_HydrophobicityD2001,_HydrophobicityD2025,_HydrophobicityD2050,_HydrophobicityD2075,_HydrophobicityD2100,_HydrophobicityD3001,_HydrophobicityD3025,_HydrophobicityD3050,_HydrophobicityD3075,_HydrophobicityD3100
0,sp|P40505|61-130,0,QYARQVRDLEEERDLELVRLRLFEEYRVSRSGIEFQEDIEKAKAEH...,70,8610.7265,0.071429,5.957659,4.286,11.429,0.000,...,2.857,37.143,45.714,65.714,85.714,8.571,25.714,47.143,74.286,97.143
1,sp|P40505|131-135,0,ERLLM,5,660.8262,0.000000,6.101802,0.000,20.000,0.000,...,0.000,0.000,0.000,0.000,0.000,60.000,100.000,60.000,80.000,100.000
2,sp|Q6GIL6|1-70,0,MRTPIIAGNWKMNKTVQEAKDFVNALPTLPDSKEVESVICAPAIQL...,70,7576.6359,0.057143,5.179817,12.857,1.429,5.714,...,4.286,27.143,58.571,74.286,98.571,1.429,17.143,41.429,62.857,100.000
3,sp|Q6GIL6|71-140,0,EDNGAFTGETSPVALADLGVKYVVIGHSERRELFHETDEEINKKAH...,70,7740.4796,0.057143,4.916652,7.143,4.286,2.857,...,5.714,15.714,38.571,72.857,98.571,8.571,25.714,47.143,70.000,84.286
4,sp|Q6GIL6|141-210,0,ANDVVGEQVKKAVAGLSEDQLKSVVIAYEPIWAIGTGKSSTSEDAN...,70,7443.2487,0.042857,4.575789,12.857,2.857,2.857,...,1.429,24.286,51.429,71.429,97.143,5.714,22.857,44.286,72.857,100.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2657,PA-MAP,1,LAAKLTKAATKLTAALTKLAAALT,24,2354.8707,0.000000,10.477740,37.500,0.000,0.000,...,8.333,25.000,54.167,70.833,100.000,4.167,4.167,50.000,66.667,95.833
2658,HSAFP1,1,DGVKLCDVPSGTWSGHCGSSSKCSQQCKDREHFAYGGACHYQFPSV...,54,5948.6696,0.111111,8.490825,3.704,3.704,0.000,...,3.704,22.222,37.037,66.667,83.333,5.556,14.815,50.000,85.185,100.000
2659,HSLIN06,1,EHFAYGGAKHYQFPSVKKFKKRQK,24,2910.3355,0.208333,10.218835,8.333,4.167,0.000,...,8.333,16.667,29.167,41.667,62.500,12.500,12.500,54.167,66.667,79.167
2660,Verine,1,RRRWWWWV,8,1330.5430,0.500000,11.999968,0.000,37.500,0.000,...,0.000,0.000,0.000,0.000,0.000,50.000,50.000,62.500,75.000,100.000


In [4]:
# X, i.e. the features or attributes
characters=peptides.to_numpy() 
print(characters)
#each row

[['sp|P40505|61-130' 0
  'QYARQVRDLEEERDLELVRLRLFEEYRVSRSGIEFQEDIEKAKAEHEKLIKLCKERLYSSIEQKIKKLQE'
  ... 47.143 74.286 97.143]
 ['sp|P40505|131-135' 0 'ERLLM' ... 60.0 80.0 100.0]
 ['sp|Q6GIL6|1-70' 0
  'MRTPIIAGNWKMNKTVQEAKDFVNALPTLPDSKEVESVICAPAIQLDALTTAVKEGKAQGLEIGAQNTYF'
  ... 41.429 62.857 100.0]
 ...
 ['HSLIN06' 1 'EHFAYGGAKHYQFPSVKKFKKRQK' ... 54.167 66.667 79.167]
 ['Verine' 1 'RRRWWWWV' ... 62.5 75.0 100.0]
 ['Phylloseptin-PTa' 1 'FLSLIPKIAGGIAALAKHL' ... 26.316 63.158 100.0]]


In [5]:
# y, i.e. the class attribute where 0=negative and 1=positive
type_label=peptides['Type'].to_numpy()
print(type_label)
#the type existent

[0 0 0 ... 1 1 1]


In [6]:
#common mapping
X=characters #rest of data
Y=type_label #0 or 1

In [7]:
# Just to confirm sequences are ok but otherwise too verbose to run
#for rows in X:
#    print(str(rows.item(2)))

## MERCI: motif discovery

Method for find motifs in the sequences and counting the frequencies of each given motif

In [8]:
#for TRAIN data
def call_and_parse_motif_on_train (x_data) :
    if os.path.exists("pos_train_seq_new.fasta"):
        os.remove("pos_train_seq_new.fasta")
    if os.path.exists("neg_train_seq_new.fasta"):
        os.remove("neg_train_seq_new.fasta")
    if os.path.exists("motif_output.occurrences"):
        os.remove("motif_output.occurrences")
        
    fp= open("pos_train_seq_new.fasta","a+")
    fn= open("neg_train_seq_new.fasta","a+")
    for rows in x_data:
        if(rows.item(1) == 1):
            fp.write('>')
            fp.write(str(rows.item(0)))
            fp.write('\n')
            fp.write((str(rows.item(2))) + '\n')
            #fp.write('\n')
    for rows in x_data:
        if(rows.item(1) == 0):
            fn.write('>')
            fn.write(rows.item(0))
            fn.write('\n')
            fn.write((str(rows.item(2)) )+ '\n')
            #fn.write('\n') 
    fp.close()  
    fn.close()
    
    #print("running motif script")
    #C:\Strawberry\perl\bin\perl.exe
    
    cmd_pl="perl MERCI-U.pl -p pos_train_seq_new.fasta -n neg_train_seq_new.fasta -o motif_output -k ALL"
    pl_script = subprocess.Popen(cmd_pl)
    
    pl_script.communicate()
    if pl_script.returncode == 0:
        print("Motif script end, now parsing")
    else:
        print("Error executing Perl script!!!")
        
    occ_file=f"{path}\motif_output.occurrences"
    #print(occ_file)
    f = open(occ_file, "r")
    occ_word_list = f.read().split()
    f.close()
    
    #print (x_data.shape)
    Xm=x_data[:,3:]
    #print (Xm.shape)
    N=x_data[:,0].shape
    b=np.zeros((N[0],1))
    Xm = np.hstack((b, Xm))
    i=0
    max = 0
    for x in x_data:
        name='>'+x[0]
        #print(x[0])
        m=occ_word_list.count(name)
        #print(m)
        Xm[i,0]=m
        #if (m>max):
        #    max = m
        i=i+1
    return Xm

In [9]:
#for test data
def call_and_parse_motif_on_test (x_data) :
    if os.path.exists("pos_test_seq.fasta"):
        os.remove("pos_test_seq.fasta")
    if os.path.exists("neg_test_seq.fasta"):
        os.remove("neg_test_seq.fasta")
    if os.path.exists("motif_test_output"):
        os.remove("motif_test_output")
    fp= open("pos_test_seq.fasta","a+")
    fn= open("neg_test_seq.fasta","a+")
    for rows in x_data:
        if(rows.item(1) == 1):
            fp.write('>')
            fp.write(str(rows.item(0)))
            fp.write('\n')
            fp.write((str(rows.item(2))) + '\n')
            #fp.write('\n')
    for rows in x_data:
        if(rows.item(1) == 0):
            fn.write('>')
            fn.write(rows.item(0))
            fn.write('\n')
            fn.write((str(rows.item(2)) )+ '\n')
            #fn.write('\n') 
    fp.close()  
    fn.close()
    #print("running motif script")
    cmd_pl="perl MERCI_motif_locator.pl -p pos_test_seq.fasta -n neg_test_seq.fasta -i motif_output -o motif_test_output"
    pl_script = subprocess.Popen(cmd_pl)
                     
    pl_script.communicate()
    #print("motif script end, now parsing")
    occ_file=f"{path}\motif_test_output"
    f = open(occ_file, "r")
    occ_word_list = f.read().split()
    f.close()

    #print (x_data.shape)
    Xm=x_data[:,3:]
    #print (Xm.shape)
    N=x_data[:,0].shape
    b=np.zeros((N[0],1))
    Xm = np.hstack((b, Xm))
    i=0
    max=0
    for x in x_data:
        name='>'+x[0]
        #print(x[0])
        m=occ_word_list.count(name)
        #print(m)
        Xm[i,0]=m
        #if (m>max):
        #    max = m
        i=i+1
    return Xm

# Split dataset 80/20

Train and test splitting such that 80% of the dataset goes to training and 20% to test.

In [10]:
X_tr, X_te, y_tr, y_te = train_test_split(X, Y, stratify=Y,  test_size=0.2, random_state=42, shuffle=True)
#statify para que a proporção das classe quer no treino como no teste seja igual
#aleatoriedade com shuffle
#random state 42 #divide na mesma posição é isso?
#divide sempre o dataset nos mesmo 80 e nos mesmos 20

In [11]:
print(X_te.shape)
#passamos de 2662 para 533 (são 20%) está ok, as colunas mantem
print(y_te.shape) #vais ser um array com 533 numero ou 0 ou 1 sem uma coluna propriamente dita

(533, 574)
(533,)


# Normalization

In [12]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1)) 
#normalize data to after this can compare

Training of a SVM model with stratitified k-fold sampling:

In [13]:
#SS_classifier = svm.SVC(kernel='rbf', C=150, gamma=0.05, probability=True)
SS_classifier = svm.SVC(kernel='rbf', C=150, gamma=0.05) #hiperparameters
sss = StratifiedKFold(n_splits =10, random_state=42, shuffle=True) #

In [37]:
#em todos os split vai se avaliar as métricas
sum_SS_f1=0 #metricas de avaliação
scores_ss = [] #accuracy
mccs_ss = [] #correlação de Matthews pq há um desequilíbrio nas classes binária
f1s_ss = []
n=0


In [38]:
#sss = StratifiedShuffleSplit(n_splits= 10, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(X_tr, y_tr):
    positive=0
    negative=0
    for i in range(y_tr.shape[0]):
        if (y_tr[i]):
            positive=positive+1
        else:
            negative=negative+1
    print('positive sample', positive)
    print('negative sample', negative)
    
    X_SS_train, X_SS_test, y_SS_train, y_SS_test = X_tr[train_index], X_tr[test_index], y_tr[train_index], y_tr[test_index]
    #divisao dentro do treino  de treino , validation acho eu
    #counts motifs in sequence and adds an additional column to the training set
    X_SS_train_new=call_and_parse_motif_on_train(X_SS_train)
    X_SS_train_new = min_max_scaler.fit_transform(X_SS_train_new)
    
    #counts motifs in sequence and adds an additional column to the test set 
    X_SS_test_new=call_and_parse_motif_on_test(X_SS_test)
    #X_SS_test_new = normalize (X_SS_test_new, max_c, min_c)
    X_SS_test_new = min_max_scaler.transform(X_SS_test_new)
    #print("test size after motif adding" , X_SS_test_new.shape)
    n=n+1
    SS_classifier.fit(X_SS_train_new, y_SS_train)
    scores_ss.append(SS_classifier.score(X_SS_test_new, y_SS_test))
    ypred=(SS_classifier.predict(X_SS_test_new))
    mcc=matthews_corrcoef(y_SS_test, ypred)
    print("MCC", mcc)
    mccs_ss.append (mcc)
    #print("accuracy", (SS_classifier.score(X_SS_test_new, y_SS_test)))
    f1=f1_score(y_SS_test, ypred)
    print("F1", f1)
    f1s_ss.append(f1)
    print("*************************************")

positive sample 194
negative sample 1935
Motif script end, now parsing
MCC 0.781074803302132
F1 0.7878787878787878
*************************************
positive sample 194
negative sample 1935
Motif script end, now parsing
MCC 0.847542615169233
F1 0.8484848484848484
*************************************
positive sample 194
negative sample 1935
Motif script end, now parsing
MCC 0.781074803302132
F1 0.7878787878787878
*************************************
positive sample 194
negative sample 1935
Motif script end, now parsing
MCC 0.8195100309119416
F1 0.8333333333333333
*************************************
positive sample 194
negative sample 1935
Motif script end, now parsing
MCC 0.8266413456321215
F1 0.8421052631578947
*************************************
positive sample 194
negative sample 1935
Motif script end, now parsing
MCC 0.8859385267630794
F1 0.8947368421052632
*************************************
positive sample 194
negative sample 1935
Motif script end, now parsing
MCC 0.88

In [39]:

print("*************************************")
print("Scores: ",np.min(scores_ss), np.max(scores_ss), np.std(scores_ss))
print("F1s: ", np.min(f1s_ss), np.max(f1s_ss), np.std(f1s_ss))
print("MCCs: ", np.min(mccs_ss), np.max(mccs_ss), np.std(mccs_ss))
print ( "avg cross-validation accuracy:", (sum(scores_ss)/10))
print ( "avg cross-validation f1:", (sum(f1s_ss)/10))
print ( "avg cross-validation mcc:", (sum(mccs_ss)/10))
print("*************************************")

#SS_classifier.fit(best_X_SS_train, best_y_SS_train)

X_new = call_and_parse_motif_on_train(X_tr)
#X_new, max_c, min_c = normalize_and_get_minmax (X_new)
X_new = min_max_scaler.fit_transform(X_new)
SS_classifier.fit(X_new,y_tr)
y_tr_predict = SS_classifier.predict(X_new)

print("*************************************")

*************************************
Scores:  0.9669811320754716 0.9812206572769953 0.005347502901240782
F1s:  0.7878787878787878 0.8947368421052632 0.03603532022312803
MCCs:  0.781074803302132 0.8859385267630794 0.037564002874547975
avg cross-validation accuracy: 0.9722849676676409
avg cross-validation f1: 0.8317646974303321
avg cross-validation mcc: 0.8232699334946085
*************************************
Motif script end, now parsing
*************************************


In [40]:
print('f1 on Train set: ', f1_score(y_tr, y_tr_predict))
print('MCC on Train set: ', matthews_corrcoef(y_tr, y_tr_predict))

tn, fp, fn, tp = confusion_matrix(y_tr, y_tr_predict).ravel()
print("tn, fp, tp, fn", tn, fp, tp, fn)
specificity = tn / (tn+fp)
print('Specificity on Train set(tn / (tn+fp)): ', specificity)
sensitivity = tp / (tp+fn)
print('Sensitivity on Train set(tp / (tp+fn)): ', sensitivity)
accuracy = (tp+tn) /(tp+tn+fp+fn)
print('Accuracy on Train set: ', accuracy)

f1 on Train set:  0.9974160206718347
MCC on Train set:  0.9971617192909873
tn, fp, tp, fn 1935 0 193 1
Specificity on Train set(tn / (tn+fp)):  1.0
Sensitivity on Train set(tp / (tp+fn)):  0.9948453608247423
Accuracy on Train set:  0.9995302959135745


In [41]:
X_te_new = call_and_parse_motif_on_test(X_te)
x_test_df = pd.DataFrame(X_te_new)
x_test_df.to_csv('x_test1.csv')#sem normlizar
X_te_new = min_max_scaler.transform(X_te_new)
x_test_df = pd.DataFrame(X_te_new)
x_test_df.to_csv('x_test2.csv')
y_SS_pred=SS_classifier.predict(X_te_new)

#TESTING NEW CODE
#for i in range(X_te_new.shape[0]):
#    if (y_te[i]):
#        if (y_SS_pred[i] != y_te[i]):
# print ("Wrong prediction for object: ", i)
#            results = SS_classifier.predict_proba(X_te_new)[i]
# print ("Actual: ", y_te[i])
# print ("Predtiction: ", y_SS_pred[i])
# gets a dictionary of {'class_name': probability}
#            prob_per_class_dictionary = dict(zip(SS_classifier.classes_, results))
# print (prob_per_class_dictionary)

#for i in range(X_te_new.shape[0]):
#    if (y_te[i]):
#        if (y_SS_pred[i] == y_te[i]):
#            print ("Correct prediction for object: ", i)
#            results = SS_classifier.predict_proba(X_te_new)[i]
#            print ("Actual: ", y_te[i])
#            print ("Predtiction: ", y_SS_pred[i])
#            # gets a dictionary of {'class_name': probability}
#            prob_per_class_dictionary = dict(zip(SS_classifier.classes_, results))
#            print (prob_per_class_dictionary)
#TESTING NEW CODE


In [42]:

print("*************************************")
print('f1 on Test set: ', f1_score(y_te, y_SS_pred))
print('MCC on Test set: ', matthews_corrcoef(y_te, y_SS_pred))
tn, fp, fn, tp = confusion_matrix(y_te, y_SS_pred).ravel()
print("tn, fp, tp, fn", tn, fp, tp, fn)
specificity = tn / (tn+fp)
print('Specificity on Test set(tn / (tn+fp)): ', specificity)
sensitivity = tp / (tp+fn)
print('Sensitivity on Test set(tp / (tp+fn)): ', sensitivity)
accuracy = (tp+tn) /(tp+tn+fp+fn)
print('Accuracy on Test set: ', accuracy)

filename = 'finalized_model_without_probab.sav'
pickle.dump(SS_classifier, open(filename, 'wb'))

*************************************
f1 on Test set:  0.9111111111111111
MCC on Test set:  0.9053554370794209
tn, fp, tp, fn 484 1 41 7
Specificity on Test set(tn / (tn+fp)):  0.9979381443298969
Sensitivity on Test set(tp / (tp+fn)):  0.8541666666666666
Accuracy on Test set:  0.9849906191369606


# Random Forest nestimators=100


In [50]:
RF_classifier = RandomForestClassifier(n_estimators=100, random_state=42)


In [59]:
#em todos os split vai se avaliar as métricas
sum_SS_f1=0 #metricas de avaliação
scores_rf = [] #accuracy
mccs_rf = [] #correlação de Matthews pq há um desequilíbrio nas classes binária
f1s_rf = []
n=0

In [51]:
for train_index, test_index in sss.split(X_tr, y_tr):
    positive = 0
    negative = 0
    for i in range(y_tr.shape[0]):
        if y_tr[i] == 1:
            positive += 1
        else:
            negative += 1
    print('Positive samples:', positive)
    print('Negative samples:', negative)
    
    X_RF_train, X_RF_test, y_RF_train, y_RF_test = X_tr[train_index], X_tr[test_index], y_tr[train_index], y_tr[test_index]
    
    # Counts motifs in sequence and adds an additional column to the training set
    X_RF_train_new = call_and_parse_motif_on_train(X_RF_train)
    X_RF_train_new = min_max_scaler.fit_transform(X_RF_train_new)
    
    # Counts motifs in sequence and adds an additional column to the test set
    X_RF_test_new = call_and_parse_motif_on_test(X_RF_test)
    X_RF_test_new = min_max_scaler.transform(X_RF_test_new)
    
    RF_classifier.fit(X_RF_train_new, y_RF_train)
    scores_rf.append(RF_classifier.score(X_RF_test_new, y_RF_test))
    y_pred = RF_classifier.predict(X_RF_test_new)
    mcc = matthews_corrcoef(y_RF_test, y_pred)
    print("MCC:", mcc)
    mccs_rf.append(mcc)
    f1 = f1_score(y_RF_test, y_pred)
    print("F1:", f1)
    f1s_rf.append(f1)
    print("*************************************")


Positive samples: 194
Negative samples: 1935
Motif script end, now parsing
MCC: 0.7807583881356304
F1: 0.7741935483870968
*************************************
Positive samples: 194
Negative samples: 1935
Motif script end, now parsing
MCC: 0.8795025417206831
F1: 0.8823529411764706
*************************************
Positive samples: 194
Negative samples: 1935
Motif script end, now parsing
MCC: 0.8146681597597366
F1: 0.8125000000000001
*************************************
Positive samples: 194
Negative samples: 1935
Motif script end, now parsing
MCC: 0.781074803302132
F1: 0.7878787878787878
*************************************
Positive samples: 194
Negative samples: 1935
Motif script end, now parsing
MCC: 0.847542615169233
F1: 0.8484848484848484
*************************************
Positive samples: 194
Negative samples: 1935
Motif script end, now parsing
MCC: 0.8859385267630794
F1: 0.8947368421052632
*************************************
Positive samples: 194
Negative samples: 19

In [52]:
print("*************************************")
print("Scores: ", np.min(scores_rf), np.max(scores_rf), np.std(scores_rf))
print("F1s: ", np.min(f1s_rf), np.max(f1s_rf), np.std(f1s_rf))
print("MCCs: ", np.min(mccs_rf), np.max(mccs_rf), np.std(mccs_rf))
print("avg cross-validation accuracy:", (sum(scores_rf) / 10))
print("avg cross-validation f1:", (sum(f1s_rf) / 10))
print("avg cross-validation mcc:", (sum(mccs_rf) / 10))
print("*************************************")


X_new = call_and_parse_motif_on_train(X_tr)
X_new = min_max_scaler.fit_transform(X_new)
RF_classifier.fit(X_new, y_tr)
y_tr_predict = RF_classifier.predict(X_new)

print("*************************************")


*************************************
Scores:  0.9624413145539906 0.9858490566037735 0.006277391757508224
F1s:  0.7499999999999999 0.9142857142857143 0.0441758622230055
MCCs:  0.7590252979875405 0.9106129237415148 0.042971911441503925
avg cross-validation accuracy: 1.9459872442200368
avg cross-validation f1: 1.6629138480150913
avg cross-validation mcc: 1.6539723647246891
*************************************
Motif script end, now parsing
*************************************


In [53]:
print('f1 on Train set: ', f1_score(y_tr, y_tr_predict))
print('MCC on Train set: ', matthews_corrcoef(y_tr, y_tr_predict))

tn, fp, fn, tp = confusion_matrix(y_tr, y_tr_predict).ravel()
print("tn, fp, tp, fn", tn, fp, tp, fn)
specificity = tn / (tn+fp)
print('Specificity on Train set(tn / (tn+fp)): ', specificity)
sensitivity = tp / (tp+fn)
print('Sensitivity on Train set(tp / (tp+fn)): ', sensitivity)
accuracy = (tp+tn) /(tp+tn+fp+fn)
print('Accuracy on Train set: ', accuracy)

f1 on Train set:  1.0
MCC on Train set:  1.0
tn, fp, tp, fn 1935 0 194 0
Specificity on Train set(tn / (tn+fp)):  1.0
Sensitivity on Train set(tp / (tp+fn)):  1.0
Accuracy on Train set:  1.0


In [57]:
X_te_new = call_and_parse_motif_on_test(X_te)
x_test_df = pd.DataFrame(X_te_new)
x_test_df.to_csv('x_test1.csv')#sem normlizar
X_te_new = min_max_scaler.transform(X_te_new)
x_test_df = pd.DataFrame(X_te_new)
x_test_df.to_csv('x_test2.csv')
y_RF_pred=RF_classifier.predict(X_te_new)

In [58]:
print("*************************************")
print('f1 on Test set: ', f1_score(y_te, y_RF_pred))
print('MCC on Test set: ', matthews_corrcoef(y_te, y_RF_pred))
tn, fp, fn, tp = confusion_matrix(y_te, y_RF_pred).ravel()
print("tn, fp, tp, fn", tn, fp, tp, fn)
specificity = tn / (tn+fp)
print('Specificity on Test set(tn / (tn+fp)): ', specificity)
sensitivity = tp / (tp+fn)
print('Sensitivity on Test set(tp / (tp+fn)): ', sensitivity)
accuracy = (tp+tn) /(tp+tn+fp+fn)
print('Accuracy on Test set: ', accuracy)

filename = 'finalized_model_without_probab.sav'
pickle.dump(RF_classifier, open(filename, 'wb'))

*************************************
f1 on Test set:  0.9090909090909091
MCC on Test set:  0.9054339748948275
tn, fp, tp, fn 485 0 40 8
Specificity on Test set(tn / (tn+fp)):  1.0
Sensitivity on Test set(tp / (tp+fn)):  0.8333333333333334
Accuracy on Test set:  0.9849906191369606


# Naive Bayes

In [1]:
from sklearn.naive_bayes import GaussianNB

# Crie o classificador Naive Bayes
NB_classifier = GaussianNB()

In [14]:
#em todos os split vai se avaliar as métricas
sum_nb_f1=0 #metricas de avaliação
scores_nb = [] #accuracy
mccs_nb = [] #correlação de Matthews pq há um desequilíbrio nas classes binária
f1s_nb = []
n=0

In [15]:
from sklearn.naive_bayes import GaussianNB



for train_index, test_index in sss.split(X_tr, y_tr):
    positive = 0
    negative = 0
    for i in range(y_tr.shape[0]):
        if y_tr[i] == 1:
            positive += 1
        else:
            negative += 1
    print('Positive samples:', positive)
    print('Negative samples:', negative)
    
    X_NB_train, X_NB_test, y_NB_train, y_NB_test = X_tr[train_index], X_tr[test_index], y_tr[train_index], y_tr[test_index]
    
    # Counts motifs in sequence and adds an additional column to the training set
    X_NB_train_new = call_and_parse_motif_on_train(X_NB_train)
    X_NB_train_new = min_max_scaler.fit_transform(X_NB_train_new)
    
    # Counts motifs in sequence and adds an additional column to the test set
    X_NB_test_new = call_and_parse_motif_on_test(X_NB_test)
    X_NB_test_new = min_max_scaler.transform(X_NB_test_new)
    
    NB_classifier.fit(X_NB_train_new, y_NB_train)
    scores_nb.append(NB_classifier.score(X_NB_test_new, y_NB_test))
    y_pred = NB_classifier.predict(X_NB_test_new)
    mcc = matthews_corrcoef(y_NB_test, y_pred)
    print("MCC:", mcc)
    mccs_nb.append(mcc)
    f1 = f1_score(y_NB_test, y_pred)
    print("F1:", f1)
    f1s_nb.append(f1)
    print("*************************************")


Positive samples: 194
Negative samples: 1935
Motif script end, now parsing
MCC: 0.4482356086523409
F1: 0.49056603773584906
*************************************
Positive samples: 194
Negative samples: 1935
Motif script end, now parsing
MCC: 0.6233654873599985
F1: 0.6521739130434783
*************************************
Positive samples: 194
Negative samples: 1935
Motif script end, now parsing
MCC: 0.4482356086523409
F1: 0.49056603773584906
*************************************
Positive samples: 194
Negative samples: 1935
Motif script end, now parsing
MCC: 0.6181599577024444
F1: 0.6399999999999999
*************************************
Positive samples: 194
Negative samples: 1935
Motif script end, now parsing
MCC: 0.5481499551970149
F1: 0.5833333333333334
*************************************
Positive samples: 194
Negative samples: 1935
Motif script end, now parsing
MCC: 0.5746354438293783
F1: 0.5964912280701754
*************************************
Positive samples: 194
Negative samples

In [16]:
print("*************************************")
print("Scores: ", np.min(scores_nb), np.max(scores_nb), np.std(scores_nb))
print("F1s: ", np.min(f1s_nb), np.max(f1s_nb), np.std(f1s_nb))
print("MCCs: ", np.min(mccs_nb), np.max(mccs_nb), np.std(mccs_nb))
print("avg cross-validation accuracy:", (sum(scores_nb) / 10))
print("avg cross-validation f1:", (sum(f1s_nb) / 10))
print("avg cross-validation mcc:", (sum(mccs_nb) / 10))
print("*************************************")

X_new = call_and_parse_motif_on_train(X_tr)
X_new = min_max_scaler.fit_transform(X_new)
NB_classifier.fit(X_new, y_tr)
y_tr_predict = NB_classifier.predict(X_new)

print("*************************************")


*************************************
Scores:  0.8732394366197183 0.9248826291079812 0.01704442051012662
F1s:  0.49056603773584906 0.6521739130434783 0.056915537014944693
MCCs:  0.4482356086523409 0.6233654873599985 0.06304488033952133
avg cross-validation accuracy: 0.9027748250509345
avg cross-validation f1: 0.5948816824428489
avg cross-validation mcc: 0.5638290877408727
*************************************
Motif script end, now parsing
*************************************


In [17]:
print('f1 on Train set: ', f1_score(y_tr, y_tr_predict))
print('MCC on Train set: ', matthews_corrcoef(y_tr, y_tr_predict))

tn, fp, fn, tp = confusion_matrix(y_tr, y_tr_predict).ravel()
print("tn, fp, tp, fn", tn, fp, tp, fn)
specificity = tn / (tn+fp)
print('Specificity on Train set(tn / (tn+fp)): ', specificity)
sensitivity = tp / (tp+fn)
print('Sensitivity on Train set(tp / (tp+fn)): ', sensitivity)
accuracy = (tp+tn) /(tp+tn+fp+fn)
print('Accuracy on Train set: ', accuracy)

f1 on Train set:  0.696588868940754
MCC on Train set:  0.6983970839807544
tn, fp, tp, fn 1766 169 194 0
Specificity on Train set(tn / (tn+fp)):  0.9126614987080104
Sensitivity on Train set(tp / (tp+fn)):  1.0
Accuracy on Train set:  0.9206200093940817


In [18]:
X_te_new = call_and_parse_motif_on_test(X_te)
x_test_df = pd.DataFrame(X_te_new)
x_test_df.to_csv('x_test1.csv')  # sem normalizar

X_te_new = min_max_scaler.transform(X_te_new)
x_test_df = pd.DataFrame(X_te_new)
x_test_df.to_csv('x_test2.csv')  # normalizado

y_NB_pred = NB_classifier.predict(X_te_new)


In [19]:
print("*************************************")
print('f1 on Test set: ', f1_score(y_te, y_NB_pred))
print('MCC on Test set: ', matthews_corrcoef(y_te, y_NB_pred))
tn, fp, fn, tp = confusion_matrix(y_te, y_NB_pred).ravel()
print("tn, fp, tp, fn", tn, fp, tp, fn)
specificity = tn / (tn + fp)
print('Specificity on Test set(tn / (tn+fp)): ', specificity)
sensitivity = tp / (tp + fn)
print('Sensitivity on Test set(tp / (tp+fn)): ', sensitivity)
accuracy = (tp + tn) / (tp + tn + fp + fn)
print('Accuracy on Test set: ', accuracy)

filename = 'finalized_model_NB.sav'
pickle.dump(NB_classifier, open(filename, 'wb'))

*************************************
f1 on Test set:  0.6131386861313869
MCC on Test set:  0.5972228106372099
tn, fp, tp, fn 438 47 42 6
Specificity on Test set(tn / (tn+fp)):  0.9030927835051547
Sensitivity on Test set(tp / (tp+fn)):  0.875
Accuracy on Test set:  0.900562851782364


# Decision tree

In [22]:
from sklearn.tree import DecisionTreeClassifier

DT_classifier = DecisionTreeClassifier(random_state=42)
#em todos os split vai se avaliar as métricas
sum_dt_f1=0 #metricas de avaliação
scores_dt = [] #accuracy
mccs_dt = [] #correlação de Matthews pq há um desequilíbrio nas classes binária
f1s_dt = []

In [23]:
for train_index, test_index in sss.split(X_tr, y_tr):
    positive = 0
    negative = 0
    for i in range(y_tr.shape[0]):
        if y_tr[i] == 1:
            positive += 1
        else:
            negative += 1
    print('Positive samples:', positive)
    print('Negative samples:', negative)
    
    X_DT_train, X_DT_test, y_DT_train, y_DT_test = X_tr[train_index], X_tr[test_index], y_tr[train_index], y_tr[test_index]
    
    # Counts motifs in sequence and adds an additional column to the training set
    X_DT_train_new = call_and_parse_motif_on_train(X_DT_train)
    X_DT_train_new = min_max_scaler.fit_transform(X_DT_train_new)
    
    # Counts motifs in sequence and adds an additional column to the test set
    X_DT_test_new = call_and_parse_motif_on_test(X_DT_test)
    X_DT_test_new = min_max_scaler.transform(X_DT_test_new)
    
    DT_classifier.fit(X_DT_train_new, y_DT_train)
    scores_dt.append(DT_classifier.score(X_DT_test_new, y_DT_test))
    y_pred = DT_classifier.predict(X_DT_test_new)
    mcc = matthews_corrcoef(y_DT_test, y_pred)
    print("MCC:", mcc)
    mccs_dt.append(mcc)
    f1 = f1_score(y_DT_test, y_pred)
    print("F1:", f1)
    f1s_dt.append(f1)
    print("*************************************")

Positive samples: 194
Negative samples: 1935
Motif script end, now parsing
MCC: 0.6712397579529541
F1: 0.6666666666666666
*************************************
Positive samples: 194
Negative samples: 1935
Motif script end, now parsing
MCC: 0.44916842875170565
F1: 0.4533333333333333
*************************************
Positive samples: 194
Negative samples: 1935
Motif script end, now parsing
MCC: 0.40565262817976494
F1: 0.40963855421686746
*************************************
Positive samples: 194
Negative samples: 1935
Motif script end, now parsing
MCC: 0.38470256194069535
F1: 0.4
*************************************
Positive samples: 194
Negative samples: 1935
Motif script end, now parsing
MCC: 0.45515393843475976
F1: 0.45945945945945943
*************************************
Positive samples: 194
Negative samples: 1935
Motif script end, now parsing
MCC: 0.474192613658045
F1: 0.49275362318840576
*************************************
Positive samples: 194
Negative samples: 1935
Moti

In [24]:
print("*************************************")
print("Scores: ", np.min(scores_dt), np.max(scores_dt), np.std(scores_dt))
print("F1s: ", np.min(f1s_dt), np.max(f1s_dt), np.std(f1s_dt))
print("MCCs: ", np.min(mccs_dt), np.max(mccs_dt), np.std(mccs_dt))
print("avg cross-validation accuracy:", (sum(scores_dt) / 10))
print("avg cross-validation f1:", (sum(f1s_dt) / 10))
print("avg cross-validation mcc:", (sum(mccs_dt) / 10))
print("*************************************")

X_new = call_and_parse_motif_on_train(X_tr)
X_new = min_max_scaler.fit_transform(X_new)
DT_classifier.fit(X_new, y_tr)
y_tr_predict = DT_classifier.predict(X_new)


*************************************
Scores:  0.7699530516431925 0.9530516431924883 0.04968416296773573
F1s:  0.37837837837837834 0.6666666666666666 0.07643254051820599
MCCs:  0.3303858343438636 0.6712397579529541 0.08637744610232846
avg cross-validation accuracy: 0.817291168394012
avg cross-validation f1: 0.4654282382158318
avg cross-validation mcc: 0.45343545948784447
*************************************
Motif script end, now parsing


In [25]:
print('f1 on Train set: ', f1_score(y_tr, y_tr_predict))
print('MCC on Train set: ', matthews_corrcoef(y_tr, y_tr_predict))

tn, fp, fn, tp = confusion_matrix(y_tr, y_tr_predict).ravel()
print("tn, fp, tp, fn", tn, fp, tp, fn)
specificity = tn / (tn+fp)
print('Specificity on Train set(tn / (tn+fp)): ', specificity)
sensitivity = tp / (tp+fn)
print('Sensitivity on Train set(tp / (tp+fn)): ', sensitivity)
accuracy = (tp+tn) /(tp+tn+fp+fn)
print('Accuracy on Train set: ', accuracy)

f1 on Train set:  1.0
MCC on Train set:  1.0
tn, fp, tp, fn 1935 0 194 0
Specificity on Train set(tn / (tn+fp)):  1.0
Sensitivity on Train set(tp / (tp+fn)):  1.0
Accuracy on Train set:  1.0


In [26]:
X_te_new = call_and_parse_motif_on_test(X_te)
x_test_df = pd.DataFrame(X_te_new)
x_test_df.to_csv('x_test1.csv')  # sem normalizar

X_te_new = min_max_scaler.transform(X_te_new)
x_test_df = pd.DataFrame(X_te_new)
x_test_df.to_csv('x_test2.csv')  # normalizado

y_DT_pred = DT_classifier.predict(X_te_new)

In [27]:
print("*************************************")
print('f1 on Test set: ', f1_score(y_te, y_DT_pred))
print('MCC on Test set: ', matthews_corrcoef(y_te, y_DT_pred))
tn, fp, fn, tp = confusion_matrix(y_te, y_DT_pred).ravel()
print("tn, fp, tp, fn", tn, fp, tp, fn)
specificity = tn / (tn + fp)
print('Specificity on Test set(tn / (tn+fp)): ', specificity)
sensitivity = tp / (tp + fn)
print('Sensitivity on Test set(tp / (tp+fn)): ', sensitivity)
accuracy = (tp + tn) / (tp + tn + fp + fn)
print('Accuracy on Test set: ', accuracy)

filename = 'finalized_model_NB.sav'
pickle.dump(DT_classifier, open(filename, 'wb'))

*************************************
f1 on Test set:  0.48000000000000004
MCC on Test set:  0.4701806142978604
tn, fp, tp, fn 400 85 42 6
Specificity on Test set(tn / (tn+fp)):  0.8247422680412371
Sensitivity on Test set(tp / (tp+fn)):  0.875
Accuracy on Test set:  0.8292682926829268


# Logistic Regression

In [28]:
from sklearn.linear_model import LogisticRegression

LR_classifier = LogisticRegression(random_state=42)

In [ ]:
#em todos os split vai se avaliar as métricas
sum_lr_f1=0 #metricas de avaliação
scores_lr = [] #accuracy
mccs_lr = [] #correlação de Matthews pq há um desequilíbrio nas classes binária
f1s_lr = []

In [ ]:
for train_index, test_index in sss.split(X_tr, y_tr):
    positive = 0
    negative = 0
    for i in range(y_tr.shape[0]):
        if y_tr[i] == 1:
            positive += 1
        else:
            negative += 1
    print('Positive samples:', positive)
    print('Negative samples:', negative)
    
    X_LR_train, X_LR_test, y_LR_train, y_LR_test = X_tr[train_index], X_tr[test_index], y_tr[train_index], y_tr[test_index]
    
    # Counts motifs in sequence and adds an additional column to the training set
    X_LR_train_new = call_and_parse_motif_on_train(X_LR_train)
    X_LR_train_new = min_max_scaler.fit_transform(X_LR_train_new)
    
    # Counts motifs in sequence and adds an additional column to the test set
    X_LR_test_new = call_and_parse_motif_on_test(X_LR_test)
    X_LR_test_new = min_max_scaler.transform(X_LR_test_new)
    
    LR_classifier.fit(X_LR_train_new, y_LR_train)
    scores_lr.append(LR_classifier.score(X_LR_test_new, y_LR_test))
    y_pred = LR_classifier.predict(X_LR_test_new)
    mcc = matthews_corrcoef(y_LR_test, y_pred)
    print("MCC:", mcc)
    mccs_lr.append(mcc)
    f1 = f1_score(y_LR_test, y_pred)
    print("F1:", f1)
    f1s_lr.append(f1)
    print("*************************************")

In [ ]:
print("*************************************")
print("Scores: ", np.min(scores_lr), np.max(scores_lr), np.std(scores_lr))
print("F1s: ", np.min(f1s_lr), np.max(f1s_lr), np.std(f1s_lr))
print("MCCs: ", np.min(mccs_lr), np.max(mccs_lr), np.std(mccs_lr))
print("avg cross-validation accuracy:", (sum(scores_lr) / 10))
print("avg cross-validation f1:", (sum(f1s_lr) / 10))
print("avg cross-validation mcc:", (sum(mccs_lr) / 10))
print("*************************************")

X_new = call_and_parse_motif_on_train(X_tr)
X_new = min_max_scaler.fit_transform(X_new)
LR_classifier.fit(X_new, y_tr)
y_tr_predict = LR_classifier.predict(X_new)


In [ ]:
print('f1 on Train set: ', f1_score(y_tr, y_tr_predict))
print('MCC on Train set: ', matthews_corrcoef(y_tr, y_tr_predict))

tn, fp, fn, tp = confusion_matrix(y_tr, y_tr_predict).ravel()
print("tn, fp, tp, fn", tn, fp, tp, fn)
specificity = tn / (tn+fp)
print('Specificity on Train set(tn / (tn+fp)): ', specificity)
sensitivity = tp / (tp+fn)
print('Sensitivity on Train set(tp / (tp+fn)): ', sensitivity)
accuracy = (tp+tn) /(tp+tn+fp+fn)
print('Accuracy on Train set: ', accuracy)

In [ ]:
X_te_new = call_and_parse_motif_on_test(X_te)
x_test_df = pd.DataFrame(X_te_new)
x_test_df.to_csv('x_test1.csv')  # sem normalizar

X_te_new = min_max_scaler.transform(X_te_new)
x_test_df = pd.DataFrame(X_te_new)
x_test_df.to_csv('x_test2.csv')  # normalizado

y_LR_pred = LR_classifier.predict(X_te_new)


In [ ]:
print("*************************************")
print('f1 on Test set: ', f1_score(y_te, y_LR_pred))
print('MCC on Test set: ', matthews_corrcoef(y_te, y_LR_pred))
tn, fp, fn, tp = confusion_matrix(y_te, y_LR_pred).ravel()
print("tn, fp, tp, fn", tn, fp, tp, fn)
specificity = tn / (tn + fp)
print('Specificity on Test set(tn / (tn+fp)): ', specificity)
sensitivity = tp / (tp + fn)
print('Sensitivity on Test set(tp / (tp+fn)): ', sensitivity)
accuracy = (tp + tn) / (tp + tn + fp + fn)
print('Accuracy on Test set: ', accuracy)

filename = 'finalized_model_NB.sav'
pickle.dump(LR_classifier, open(filename, 'wb'))

# ROC curve

In [61]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# SVM
y_svm_pred_prob = SS_classifier.decision_function(X_te)
fpr_svm, tpr_svm, _ = roc_curve(y_te, y_svm_pred_prob)
auc_svm = roc_auc_score(y_te, y_svm_pred_prob)

# Random Forest
y_rf_pred_prob = RF_classifier.predict_proba(X_te)[:, 1]
fpr_rf, tpr_rf, _ = roc_curve(y_te, y_rf_pred_prob)
auc_rf = roc_auc_score(y_te, y_rf_pred_prob)

# Plotando as curvas ROC
plt.plot(fpr_svm, tpr_svm, label='SVM (AUC = %0.2f)' % auc_svm)
plt.plot(fpr_rf, tpr_rf, label='Random Forest (AUC = %0.2f)' % auc_rf)
plt.plot([0, 1], [0, 1], 'k--')  # Linha diagonal para referência
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()


ValueError: could not convert string to float: 'EUF06718.1'